In [88]:
from bs4 import BeautifulSoup
import requests
import re
import json
import urllib
import logging
from dataclasses import dataclass
from collections import Counter

In [91]:
logging.basicConfig(level=logging.DEBUG)

In [92]:
@dataclass
class CODE:
    SUCCESS:int=200

In [93]:
class NovelWebsite:
    def __init__(self,domain) -> None:
        self.domain=domain
    
    def search_url(self,keyword):
        return urllib.parse.urljoin(self.domain,f"search?keyword={urllib.parse.quote(keyword)}")
    
    def search_raw(self,keyword):
        data=requests.get(self.search_url(keyword))
        return data
    def join(self,url):
        return urllib.parse.urljoin(self.domain,url)

    def search_result(self,keyword):
        result=self.search_raw(keyword)
        logging.debug("STATUS_CODE: "+str(result.status_code))
        if result.status_code==CODE.SUCCESS:
            raw_page=BeautifulSoup(result.content)
            data=raw_page.find("div",{"id":"main"})
            return [{"name":data.a.contents[0].strip(),"url":self.join(data.a.attrs.get("href"))}]


In [136]:
urllib.parse.urlparse(novel.url).netloc

ParseResult(scheme='https', netloc='www.9biqu.com', path='/biquge/6027/', params='', query='', fragment='')

In [190]:
class Novel:
    def __init__(self,name,url) -> None:
        self.name=name
        self.url=url
        parse_res=urllib.parse.urlparse(self.url)
        self.domain=parse_res.scheme+"://"+parse_res.netloc
        self.Update()
        #TODO: async update

    def Update(self):
        raw_result=requests.get(self.url)
        if raw_result.status_code==CODE.SUCCESS:
            self.content=BeautifulSoup(raw_result.content)
        self.contentTable=self.GetContentTable()

    def GetContentTable(self):
        res=self.content.find_all("dd")
        counter=Counter(res)
        while(counter[res[0]]>1): # Remove duplicate newest chapters
            res.pop(0)
        res=list(map(lambda x:{"name":x.a.contents[0].strip(),"url": urllib.parse.urljoin(self.domain,x.a.attrs.get("href"))},res))
        return res
    def GetChapter(self,idx):
        return self.contentTable[idx]

In [191]:
biquge=NovelWebsite("https://www.9biqu.com/")
books=biquge.search_result("灵境行者")
novel=Novel(**books[0])
# contentTable=novel.GetContentTable()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.9biqu.com:443
DEBUG:urllib3.connectionpool:https://www.9biqu.com:443 "GET /search?keyword=%E7%81%B5%E5%A2%83%E8%A1%8C%E8%80%85 HTTP/1.1" 200 None
DEBUG:root:STATUS_CODE: 200
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.9biqu.com:443
DEBUG:urllib3.connectionpool:https://www.9biqu.com:443 "GET /biquge/6027/ HTTP/1.1" 200 None


In [193]:
url=novel.GetChapter(0)["url"]
raw_res=requests.get(url)
if raw_res.status_code==CODE.SUCCESS:
    raw_content=BeautifulSoup(raw_res.content)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.9biqu.com:443
DEBUG:urllib3.connectionpool:https://www.9biqu.com:443 "GET /biquge/6027/15056952.html HTTP/1.1" 200 None


In [199]:
raw_page_content=raw_content.find_all("div",{"id":"content"})[0].find_all("p")
raw_page_content.pop(0)

<p>
                                天才一秒记住本站地址：[
                            笔趣阁]
                            https://www.9biqu.com最快更新！无广告！<br/><br/>
</p>

In [216]:
list(map(lambda x:x.replace("一秒记住www.9biqu.com","").strip() ,list(map(lambda x:x.contents[0].strip(),raw_page_content))))

['松海市。',
 '早上七点半，昏暗的房间里，松软的大床上，张元清陡然惊醒，捂着头，弓身如虾。',
 '头痛的像是要裂开，脑袋里如同嵌了钢针，疼的头皮都在抽搐，以至于产生了幻听、幻觉，脑海里闪过纷乱的画面，耳边尽是嘈杂的、没有意义的噪音。',
 '张元清知道旧疾复发了。',
 '颤巍巍的从床上爬起来，拉开床头柜的抽屉，哆哆嗦嗦的摸到药瓶，迫不及待的拧开，倒了五六颗蓝色小药丸，囫囵吞下。',
 '然后，他把自己摔回床上，大口喘息，忍耐剧痛。',
 '十几秒后，撕裂灵魂的头痛减弱，继而平复。',
 '“呼......”张元清如释重负的吐出一口气，满头冷汗。',
 '他读高中时得了一场怪病，症状是大脑不受控制的回忆起过往的所有记忆，包括被遗忘的垃圾信息；不受控制的采集外界信息，进行分析；大脑对身体的掌控达到一个不可思议的程度。',
 '幸运的是，这种状态无法持续太久，就会因为身体不堪重负而中断。',
 '正是因为这种能力，他玩儿似的考上松海大学——全国排名前列的名牌学府。',
 '张元清把这种状态称为大脑过载，他认为自己可能要进化成超人了，但因为身体无法支撑这种进化，才频频中断。',
 '当他把这个猜测告诉医生时，医生表示听不懂，但大受震撼，并建议他去楼下的精神科看看。',
 '总之医院也查不出病因，后来，老妈从国外给他带回来了特效药，病情这才得到控制，只要定期吃药，就不会发作。',
 '“一准是昨晚没休息好，太累了，都怪江玉饵，大半夜的非要来我房间打游戏......”',
 '嘴上虽然这么说，但内心却悄然沉重，因为张元清知道，药效的作用开始减弱，自己的病症越来越严重了。',
 '“以后要加大药量了.......”张元清穿上棉拖鞋，来到窗边，‘刷’的拉开帘子。',
 '阳光争先恐后的涌进来，把房间填满。',
 '松海市的四月，春光明媚，迎面而来的晨风清凉舒适。',
 '“咚咚！”',
 '这时，敲门声传来，外婆在门外喊道：',
 '“元子，起床了。”',
 '“不起！”张元清冷酷无情的拒绝，他想睡回笼觉。',
 '春光明媚，又是周末，不睡懒觉岂不是浪费人生？',
 '“给你三分钟，不起床我就泼醒你。”',
 '外婆更加冷酷无情。',
 '“知道了知道了.....”张元清立刻服软。',
 '他知道脾气暴躁的外婆真能干出这事儿。',
 '在张元清还读小学时，父亲就因车祸去世

In [183]:
biquge="https://www.9biqu.com/"

In [184]:
biquge="www.9biqu.com/search?keyword={}"
query_path=biquge.format(urllib.parse.quote(novel_name))

In [26]:
def UrlRipper(url,type="https"):
    if type=="https":
        return "https://"+url

In [29]:
page=requests.get(UrlRipper(query_path))

In [30]:
UrlRipper(query_path)

'https://www.9biqu.com/search?keyword=%E7%81%B5%E5%A2%83%E8%A1%8C%E8%80%85'

In [31]:
soup=BeautifulSoup(page.content)

In [37]:
data=soup.find("div",{"id":"main"})
data.a.attrs.get("href")

In [ ]:
class NovelFinder:
    def __init__(self) -> None:
        searchlist=["https://www.9biqu.com/search?keyword={0}"]